Classify:

In [1]:
from datetime import datetime
import ast

import numpy as np
import pandas as pd
import torch
from PIL import Image
from tqdm.auto import tqdm
from torch import tensor
from torchvision.transforms import transforms, InterpolationMode

SPECIESNET_PATH = 'models/speciesnet-pytorch-v4.0.1a-v1/'
SPECIESNET_MODEL = SPECIESNET_PATH + 'always_crop_99710272_22x8_v12_epoch_00148.pt'

LABELS_FILE = 'models/speciesnet-pytorch-v4.0.1a-v1/always_crop_99710272_22x8_v12_epoch_00148.labels.txt'

e:\mgr\.venv_windows\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup

In [ ]:
images = pd.read_csv('../megadetector_results_2.csv', index_col=0)
images['image_path'] = '../' + images['image_path']
images['bbox'] = images["bbox"].apply(
    lambda b: ast.literal_eval(b) if isinstance(b, str) else None)

In [ ]:
CROP_SIZE = 480
BATCH_SIZE = 50

with open(LABELS_FILE, "r", encoding="utf-8") as f:
    classes_labels = [line.strip() for line in f if line.strip()]


class Classifier:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = torch.load(SPECIESNET_MODEL, map_location=self.device, weights_only=False)
        self.model.eval()
        print(f'Speciesnet loaded onto {self.device}')

        # transform image to form usable by network
        self.transforms = transforms.Compose([
            transforms.Resize(size=(CROP_SIZE, CROP_SIZE), interpolation=InterpolationMode.BICUBIC),
            transforms.ToTensor()
        ])

    def predictOnBatch(self, pil_images, withsoftmax=True):
        """
        pil_images: list of PIL.Image
        returns: np.ndarray of shape (B, num_classes)
        """
        # Transform all images
        tensors = [self.transforms(im) for im in pil_images]   # list of (3, H, W)
        batch = torch.stack(tensors).to(self.device)           # (B, 3, H, W)
        batch = batch.permute(0, 2, 3, 1)                      # (B, H, W, 3)

        with torch.no_grad():
            logits = self.model(batch)
            preds = logits.softmax(dim=1) if withsoftmax else logits

        return preds.cpu()


def crop_normalized_bbox(img: Image.Image, bbox: list[float]):
    """
    img: PIL.Image opened image
    bbox: list [x, y, w, h], normalized 0-1
    returns cropped PIL.Image
    """
    W, H = img.size
    x, y, w, h = bbox

    left   = int(x * W)
    top    = int(y * H)
    right  = int((x + w) * W)
    bottom = int((y + h) * H)

    return img.crop((left, top, right, bottom))

classifier = Classifier()

Speciesnet loaded onto cuda


### The loop

In [ ]:
results = pd.DataFrame({'image': [], 'detected_animal': [], 'confidence': []})
batch = []
paths = []

for _, row in tqdm(images.iterrows(), total=len(images)):
    image_path = row['image_path']

    category = row['category']
    if category != 1:
        results.loc[len(results)] = [image_path, 'empty', 0]
        continue

    try:
        image = Image.open(image_path).convert("RGB")
        cropped_image = crop_normalized_bbox(image, row['bbox'])
    except Exception as e:
        print(f'Error in image {image_path}: {e}')
        continue

    paths.append(image_path)
    batch.append(cropped_image)

    if len(batch) == BATCH_SIZE:
        preds = classifier.predictOnBatch(batch)
        top_probs, class_indxes = preds.max(dim=1)
        confs = top_probs.cpu().numpy()
        detections = [classes_labels[idx].split(';')[-1] for idx in class_indxes]

        batch_results = pd.DataFrame(
            {'image': paths, 'detected_animal': detections, 'confidence': confs})
        results = pd.concat([results, batch_results], ignore_index=True)
        batch = []
        paths = []

if len(batch) > 0:
    preds = classifier.predictOnBatch(batch)
    top_probs, class_indxes = preds.max(dim=1)
    confs = top_probs.cpu().numpy()
    detections = [classes_labels[idx].split(';')[-1] for idx in class_indxes]

    batch_results = pd.DataFrame(
        {'image': paths, 'detected_animal': detections, 'confidence': confs})
    results = pd.concat([results, batch_results], ignore_index=True)

now = datetime.now().strftime('%Y_%m_%d_%H_%M')
results.to_csv('../results/speciesnet/results_speciesnet_' + now + '.csv')

 24%|██▍       | 4431/18107 [17:03<31:31,  7.23it/s]  

Error in image ../../pictures/10_ŁUPKÓW/B/Późna jesień/1/2022-11-18 05-58-30.JPG: broken data stream when reading image file


 24%|██▍       | 4435/18107 [17:04<36:34,  6.23it/s]

Error in image ../../pictures/10_ŁUPKÓW/B/Późna jesień/1/2022-11-18 05-59-31.JPG: broken data stream when reading image file


 25%|██▍       | 4458/18107 [17:10<45:36,  4.99it/s]  

...

 31%|███▏      | 5662/18107 [21:33<36:24,  5.70it/s]  

Error in image ../../pictures/11_ZUBEŃSKO/B/Wczesna jesień/3/2022-10-02 21-42-15.JPG: broken data stream when reading image file


 45%|████▍     | 8122/18107 [30:37<26:49,  6.20it/s]  

Error in image ../../pictures/16_HUTA_KOMOROWSKA/B/Lato/1/2023-07-08 08-12-33.JPG: image file is truncated (22 bytes not processed)


100%|██████████| 18107/18107 [1:01:04<00:00,  4.94it/s]


In [6]:
TODO OSTATNIE LINIJKI Z TEGO RESULT NIE SĄ SPLIT GATUNKI

SyntaxError: invalid syntax (3707926352.py, line 1)